## import packges

In [1]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd

# visualization
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

from joblib import Parallel, delayed

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## CONFIG

In [2]:
## CONFIG
class CFG:
    seed          = 222
    debug         = False # set debug=False for Full Training
    model_name    = 'Unet'
    backbone      = 'efficientnet-b7'
    train_bs      = 20
    valid_bs      = train_bs*2
    img_size      = [384, 384]
    epochs        = 20  if not debug else 3
    lr            = 2e-3
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = epochs+5
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = 1
    n_fold        = 5
    folds         = [0,1,2,3,4]
    num_classes   = 3
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # you need replace the base_path as your local data path
    base_path = '../input'
    pretrained = False
    model_path =base_path + '/uwmgi-pretrained-efficentb0-lr2e-3'
    log_path = './logpath/'+f'{time.strftime("%Y_%m_%d_%H_%M", time.localtime())}' +( '_debug' if debug else '')
#   2.5 d data set
    img_stride = 2
    img_channel = 3

In [3]:
print(f'Debug : {CFG.debug}')
if not os.path.exists(CFG.log_path):
    os.makedirs(CFG.log_path)
    print(f'make dir {CFG.log_path}')

Debug : False
make dir ./logpath/2022_06_29_16_30


## Set Seed

In [4]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE',seed)
    
set_seed(CFG.seed)

> SEEDING DONE 222


## rle decode and encode

In [5]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = np.asarray(mask_rle.split(), dtype=int)
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction


# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## utils

In [6]:
def id2mask(id_,df):
    idf = df[df['id']==id_]
    wh = idf[['height','width']].iloc[0]
    shape = (wh.height, wh.width, 3)
    mask = np.zeros(shape, dtype=np.uint8)
    rles =idf.segmentation.tolist()[0]
    i=0
    for rle in rles:
        mask[..., i] = rle_decode(rle, shape[:2])
        i+=1
    mask = mask.astype('float32')
    return mask

def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = img.astype('uint') # original is uint16
    return img

def load_2_5dimg(paths,h,w):
    new_img = np.zeros((h,w,CFG.img_channel), dtype=np.uint)
    for i in range(CFG.img_channel):
        new_img[..., i] += load_img(paths[i])

    new_img = new_img.astype('float32')
    mx =new_img.max()
    if mx:
        new_img = new_img /mx
    return new_img
def show_img(img, mask=None):
    plt.imshow(img, cmap='bone')
    
    if mask is not None:
        # plt.imshow(np.ma.masked_where(mask!=1, mask), alpha=0.5, cmap='autumn')
        plt.imshow(mask, alpha=0.5)
        handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
        labels = [ "Large Bowel", "Small Bowel", "Stomach"]
        plt.legend(handles,labels)
    plt.axis('off')

def plot_batch(imgs, msks, k=5):
    img_id =0
    size=imgs.shape[0]
    for idx in range(size//k ):
        plt.figure(figsize=(5*5,k))
        for i in range(k):
            plt.subplot(1, k, (i)%k +1)
            img = imgs[img_id,].permute((1, 2, 0)).numpy()*255.0
            img = img.astype('uint8')
    #         msk = msks[idx,].permute((1, 2, 0)).numpy()*255.0
            msk = msks[img_id,].permute((1, 2, 0)).numpy()*255.0
            msk = msk.astype('uint8')
            show_img(img, msk)
            img_id+=1
        plt.tight_layout()
        plt.show()

## make 2.5d data

In [7]:
def make_25data(df):
    channels = CFG.img_channel
    stride = CFG.img_stride
    for i in range(channels):
        df[f'image_path_{i:02}'] = df.groupby(['case','day'])['image_path'].shift(-i*stride).fillna(method="ffill")
    df['image_paths'] = df[[f'image_path_{i:02d}' for i in range(channels)]].values.tolist()
    for i in range(channels):
        df.drop(labels=[f'image_path_{i:02}'],axis=1,inplace=True)
    return df

## Create Folds

In [8]:
def creat_folds(df):
    skf = StratifiedGroupKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['empty'], groups = df["case"])):
        df.loc[val_idx, 'fold'] = fold
    print(df.groupby(['fold','empty'])['id'].count())
    return df

## deal csv

In [9]:
def deal_csv():
    df =pd.read_csv(CFG.base_path + '/uw-madison-gi-tract-image-segmentation/train.csv')
    df['empty'] = df.segmentation.map(lambda x: bool(pd.isna(x)))
    df['segmentation'] = df.segmentation.fillna('')
    df2 = df.groupby(['id'])['class'].agg(list).to_frame().reset_index()
    df2 = df2.merge(df.groupby(['id'])['segmentation'].agg(list), on=['id'])
    df = df.drop(columns=['segmentation', 'class'])
    df = df.groupby(['id']).head(1).reset_index(drop=True)
    df = df.merge(df2, on=['id'])
    df['case'] = df.id.map(lambda x : int(x.split('_')[0].replace('case','')) )
    df['day'] = df.id.map(lambda x : int(x.split('_')[1].replace('day','')) )
    df['slice'] = df.id.map(lambda x : int(x.split('_')[-1]) )


    paths = glob(f'{CFG.base_path}/uw-madison-gi-tract-image-segmentation/train/**/*png',recursive=True)
    path_df = pd.DataFrame(paths, columns=['image_path'])
    data = path_df['image_path'].map(lambda x: x.split('/'))
    path_df['slice'] = data.map(lambda x : int(x[-1].split('_')[1]))
    path_df['case'] = data.map(lambda x : int(x[-3].split('_')[0].replace('case','')))
    path_df['day'] = data.map(lambda x : int(x[-3].split('_')[1].replace('day','')))
    path_df['width'] = data.map(lambda x : int(x[-1].split('_')[2]))
    path_df['height'] = data.map(lambda x : int(x[-1].split('_')[3]))
    df = df.merge(path_df, on=['case','day','slice'], how='left')

    df =make_25data(df)
    df = creat_folds(df)
    return df

## Remove Faulty

In [10]:
def remove_faulty(df):
    fault1 = 'case7_day0'
    fault2 = 'case81_day30'
    df = df[~df['id'].str.contains(fault1) & ~df['id'].str.contains(fault2)].reset_index(drop=True)
    return df

## Build Dataset

In [11]:
class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=True, transforms=None,test=False):
        self.df         = df
        self.label      = label
        self.img_paths  = df['image_paths'].tolist()
        self.ids        = df['id'].tolist()
        self.widths     = df['width'].tolist()
        self.heights     = df['height'].tolist()
        self.transforms = transforms
        self.test = test
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_paths  = self.img_paths[index]
        h = self.heights[index]
        w = self.widths[index]
        img = load_2_5dimg(img_paths,h,w)
        id_ = self.ids[index]
        
        
        if self.label:
            msk = id2mask(id_ ,self.df)
            if self.transforms:
                data = self.transforms(image=img, mask=msk)
                img  = data['image']
                msk  = data['mask']

            img = np.transpose(img, (2, 0, 1))
            msk = np.transpose(msk, (2, 0, 1))
            return torch.tensor(img), torch.tensor(msk)
        else:
            if self.transforms:
                data = self.transforms(image=img)
                img  = data['image']

            img = np.transpose(img, (2, 0, 1))
            if self.test:
                return torch.tensor(img), id_, h, w
            return torch.tensor(img)

## Augmentations

In [12]:
data_transforms = {
    "train": A.Compose([
        A.PadIfNeeded(*CFG.img_size,position="center",border_mode=cv2.BORDER_CONSTANT),
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=45, p=0.6),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
# #             A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        ], p=0.33),
        # A.RandomGamma(gamma_limit=(80, 120), eps=None, p=0.6),
        ], p=1.0),
    
    "valid": A.Compose([
        A.PadIfNeeded(*CFG.img_size,position="center",border_mode=cv2.BORDER_CONSTANT),
        A.Resize(*CFG.img_size, interpolation=cv2.INTER_NEAREST),
        ], p=1.0)
}

## DataLoader

In [13]:
def prepare_loaders(df,fold, debug=False):
    train_df = df.query("fold!=@fold").reset_index(drop=True)
    valid_df = df.query("fold==@fold").reset_index(drop=True)
    if debug:
        train_df = train_df.head(32*5).query("empty==0")
        valid_df = valid_df.head(32*5).query("empty==0")
    train_dataset = BuildDataset(train_df, transforms=data_transforms['train'])
    valid_dataset = BuildDataset(valid_df, transforms=data_transforms['valid'])

    train_loader = DataLoader(train_dataset, batch_size=CFG.train_bs if not debug else 20, 
                              num_workers=2, shuffle=True, pin_memory=False, drop_last=False)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.valid_bs if not debug else 20, 
                              num_workers=2, shuffle=False, pin_memory=False)
    
    return train_loader, valid_loader

## Check Mask

In [14]:
def check_mask(df):
    train_loader, valid_loader =prepare_loaders(df,0,True)
    imges ,mask =next(iter(train_loader))
    plot_batch(imges,mask)

## Build Model

In [15]:
import segmentation_models_pytorch as smp

def build_model():
    model = smp.Unet(
        encoder_name=CFG.backbone,      # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights= 'imagenet',     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=CFG.img_channel,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=CFG.num_classes,        # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to(CFG.device)
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [16]:
# from torchsummary import summary

# model =build_model()
# summary(model.cuda() ,(CFG.img_channel,*CFG.img_size))
# del model

## Loss Function

In [17]:
JaccardLoss = smp.losses.JaccardLoss(mode='multilabel')
DiceLoss    = smp.losses.DiceLoss(mode='multilabel')
BCELoss     = smp.losses.SoftBCEWithLogitsLoss()
LovaszLoss  = smp.losses.LovaszLoss(mode='multilabel', per_image=False)
TverskyLoss = smp.losses.TverskyLoss(mode='multilabel', log_loss=False)

def dice_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean(dim=(1,0))
    return dice

def iou_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    union = (y_true + y_pred - y_true*y_pred).sum(dim=dim)
    iou = ((inter+epsilon)/(union+epsilon)).mean(dim=(1,0))
    return iou

def criterion(y_pred, y_true):
    # return 0.5*BCELoss(y_pred, y_true) + 0.5*TverskyLoss(y_pred, y_true)
    return 0.5*DiceLoss(y_pred, y_true) + 0.5*TverskyLoss(y_pred, y_true)

## Training Function

In [18]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = criterion(y_pred, masks)
            loss   = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            # if scheduler is not None:
            #     scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    
    return epoch_loss

## Validation Function

In [19]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (images, masks) in pbar:        
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        loss    = criterion(y_pred, masks)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        y_pred = nn.Sigmoid()(y_pred)
        val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        val_scores.append([val_dice, val_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        gpu_memory=f'{mem:0.2f} GB')
    val_scores  = np.mean(val_scores, axis=0)
    
    return epoch_loss, val_scores

## Run Training

In [20]:
def run_training(model, optimizer, scheduler, num_epochs ,train_loader , valid_loader,fold):
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_dice      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)

    for epoch in range(1, num_epochs + 1): 
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CFG.device, epoch=epoch)
        
        val_loss, val_scores = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch)
        val_dice, val_jaccard = val_scores
    
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Dice'].append(val_dice)
        history['Valid Jaccard'].append(val_jaccard)
        
        print(f'Valid Dice: {val_dice:0.4f} | Valid Jaccard: {val_jaccard:0.4f}')
        
        # deep copy the model
        if val_dice >= best_dice:
            print(f"{c_}Valid Score Improved ({best_dice:0.4f} ---> {val_dice:0.4f})")
            best_dice    = val_dice
            best_jaccard = val_jaccard
            best_epoch   = epoch
            
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"{CFG.log_path}//best-{fold:02d}.bin"
            torch.save(model.state_dict(),PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        last_model_wts = copy.deepcopy(model.state_dict())
        # PATH = f"last_epoch-{fold:02d}.bin"
        # h.save(model.state_dict(), PATH)  
        scheduler.step()
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Score: {:.4f}".format(best_jaccard))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    conf[f'fold{fold}_dice'] = best_dice
    conf[f'fold{fold}_jaccard'] = best_jaccard
    pd.DataFrame(history , columns =['Train Loss',
                                    'Valid Loss',
                                    'Valid Dice',
                                    'Valid Jaccard'] ).to_csv(f"{CFG.log_path}//history_fold{fold}.csv")
    return model, history

## LR scheduler

In [21]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == None:
        return None
        
    return scheduler

## Training

In [22]:
def run_train():
    df = deal_csv()
    print(f'check the transformed dataset')
    if CFG.debug:
        check_mask(df)
    print(f'Begin Training')
    if torch.cuda.is_available():
        print("Using cuda: {}\n".format(torch.cuda.get_device_name()))
    for fold in CFG.folds:
        print(f'#'*15)
        print(f'### Fold: {fold}')
        print(f'#'*15)
        train_loader, valid_loader = prepare_loaders(df,fold=fold, debug=CFG.debug)
        if not CFG.pretrained :
            model = build_model()
        else:
            path =glob(CFG.model_path+f'/*0{fold}.bin')[0]
            print(f'usign pretrained model{path}')
            model = load_model(path)
        optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
        scheduler = fetch_scheduler(optimizer)
        
        model, history = run_training(model, optimizer, scheduler,
                                    num_epochs=CFG.epochs,train_loader=train_loader,valid_loader=valid_loader,fold=fold)
        
        
    return 
    

In [23]:
dic_conf =dict(CFG.__dict__)
for key in dic_conf:
    dic_conf[key] = str(dic_conf[key])
conf= pd.DataFrame(dic_conf,index=[0])


## RUN

In [24]:
run_train()
conf.to_csv(f"{CFG.log_path}//result.csv")

fold  empty
0.0   False    2552
      True     4216
1.0   False    2420
      True     4476
2.0   False    2669
      True     4435
3.0   False    2727
      True     5129
4.0   False    3717
      True     6155
Name: id, dtype: int64
check the transformed dataset
Begin Training
Using cuda: NVIDIA TITAN RTX

###############
### Fold: 0
###############
Epoch 1/20

Valid : 100%|██████████| 170/170 [02:57<00:00,  1.04s/it, gpu_memory=8.48 GB, valid_loss=0.2441]


Valid Dice: 0.8261 | Valid Jaccard: 0.7900
Valid Score Improved (-inf ---> 0.8261)
Model Saved

Epoch 2/20

Valid : 100%|██████████| 170/170 [02:57<00:00,  1.05s/it, gpu_memory=8.59 GB, valid_loss=0.2141]


Valid Dice: 0.8601 | Valid Jaccard: 0.8258
Valid Score Improved (0.8261 ---> 0.8601)
Model Saved

Epoch 3/20

...

Valid : 100%|██████████| 197/197 [03:17<00:00,  1.00s/it, gpu_memory=9.27 GB, valid_loss=0.1326]


Valid Dice: 0.9166 | Valid Jaccard: 0.8898
Valid Score Improved (0.9144 ---> 0.9166)
Model Saved

Epoch 13/20

Valid : 100%|██████████| 197/197 [03:19<00:00,  1.01s/it, gpu_memory=9.27 GB, valid_loss=0.1530]


Valid Dice: 0.9100 | Valid Jaccard: 0.8859

Epoch 14/20

Train :  83%|████████▎ | 1264/1532 [57:34<13:07,  2.94s/it, gpu_mem=20.33 GB, lr=0.00094, train_loss=0.1820] 